![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [75]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
# read in the data
rentals = pd.read_csv('rental_info.csv', parse_dates = ['rental_date', 'return_date'])

# view the data
rentals.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


### In this project, you will use regression models to predict the number of days a customer rents DVDs for.

### As with most data science projects, you will need to pre-process the data provided, in this case, a csv file called rental_info.csv. Specifically, you need to:

* **Read in the csv file `rental_info.csv` using `pandas`.**
* **Create a column named `"rental_length_days"` using the columns `"return_date"` and `"rental_date"`, and add it to the `pandas` DataFrame. This column should contain information on how many days a DVD has been rented by a customer.**
* **Create two columns of dummy variables from `"special_features"`, which takes the value of 1 when:**
    * The value is `"Deleted Scenes"`, storing as a column called `"deleted_scenes"`.
    * The value is `"Behind the Scenes"`, storing as a column called `"behind_the_scenes"`.
* **Make a `pandas` DataFrame called `X` containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.**
* **Choose the `"rental_length_days"` as the target column and save it as a pandas Series called `y`.**

### Following the preprocessing you will need to:

* **Split the data into `X_train`, `y_train`, `X_test`, and `y_test` train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.**
* ***Set `random_state` to `9`* whenever you use a function/method involving randomness, for example, when doing a test-train split.**

### Recommend a model yielding a mean squared error (MSE) less than 3 on the test set

* **Save the model you would recommend as a variable named `best_model`, and save its MSE on the test set as `best_mse`.**

In [76]:
# make a copy of the dataset
rental = rentals.copy()

rental.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [77]:
# check the dtypes
rental.dtypes

rental_date         datetime64[ns, UTC]
return_date         datetime64[ns, UTC]
amount                          float64
release_year                    float64
rental_rate                     float64
length                          float64
replacement_cost                float64
special_features                 object
NC-17                             int64
PG                                int64
PG-13                             int64
R                                 int64
amount_2                        float64
length_2                        float64
rental_rate_2                   float64
dtype: object

In [78]:
# create a column for duration a dvd was rented in days
rental['rental_length_days'] = (rental['return_date'] - rental['rental_date']).dt.days

rental.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


In [79]:
# create dummy variables for deleted and behind the scenes
def encoder(x):
    if "Deleted Scenes" in x:
        return 1
    else: 
        return 0

# encode for deleted scenes
rental['deleted_scenes'] = rental['special_features'].apply(encoder)

# view
rental.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3,0
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7,0
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4,0


In [80]:
# encode for behind the scenes
def encoder(x):
    if "Behind the Scenes" in x:
        return 1
    else: 
        return 0

# create column
rental['behind_the_scenes'] = rental['special_features'].apply(encoder)

# view
rental[rental['special_features'] == '{"Behind the Scenes"}'].head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
181,2005-05-25 04:19:27+00:00,2005-05-30 09:44:27+00:00,4.99,2010.0,4.99,119.0,17.99,"{""Behind the Scenes""}",0,0,1,0,24.9001,14161.0,24.9001,5,0,1
182,2005-06-15 13:34:50+00:00,2005-06-21 16:07:50+00:00,4.99,2010.0,4.99,119.0,17.99,"{""Behind the Scenes""}",0,0,1,0,24.9001,14161.0,24.9001,6,0,1
183,2005-07-09 09:28:38+00:00,2005-07-10 06:46:38+00:00,4.99,2010.0,4.99,119.0,17.99,"{""Behind the Scenes""}",0,0,1,0,24.9001,14161.0,24.9001,0,0,1
184,2005-07-30 04:47:03+00:00,2005-08-06 07:23:03+00:00,4.99,2010.0,4.99,119.0,17.99,"{""Behind the Scenes""}",0,0,1,0,24.9001,14161.0,24.9001,7,0,1
185,2005-08-22 12:42:32+00:00,2005-08-28 09:04:32+00:00,4.99,2010.0,4.99,119.0,17.99,"{""Behind the Scenes""}",0,0,1,0,24.9001,14161.0,24.9001,5,0,1


In [81]:
# Create X
X = rental.drop(['rental_date', 'return_date', 'special_features', 'rental_length_days'], axis = 'columns')

X.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1


In [82]:
# create y
y = rental['rental_length_days']

y.head()

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64

In [83]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

In [84]:
# find a model yielding mse of 3 or less on test set
# begin with basic models
# imports 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [85]:
# basic dt
dt = DecisionTreeRegressor(random_state = 9)

dt.fit(X_train, y_train)

dt_pred = dt.predict(X_test)

mse_dt = mean_squared_error(y_test, dt_pred)

print('The MSE of a basic Decision Tree is {}'.format(mse_dt))

The MSE of a basic Decision Tree is 2.1675004952579413


In [86]:
# Try out basic versions other models
rf = RandomForestRegressor(random_state = 9)

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_pred)

print(f'MSE of a basic Random Forest Model is {rf_mse}')

MSE of a basic Random Forest Model is 2.030141907417274


In [87]:
# linear model
lr = LinearRegression()

lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)

lr_mse = mean_squared_error(y_test, lr_pred)

print(f'Basic Linear Model MSE is {lr_mse}')

Basic Linear Model MSE is 2.9417238646975883


In [88]:
# ada
ada = AdaBoostRegressor(base_estimator = dt, random_state = 9)

ada.fit(X_train, y_train)

ada_pred = ada.predict(X_test)

ada_mse = mean_squared_error(y_test, ada_pred)

print(f'Basic Ada Boost MSE is {ada_mse}')

Basic Ada Boost MSE is 2.031165829185687


In [89]:
# gradient
grad = GradientBoostingRegressor(random_state = 9)

grad.fit(X_train, y_train)

grad_pred = grad.predict(X_test)

grad_mse = mean_squared_error(y_test, grad_pred)

print(f'The MSE for a basic GradientBoosting Model is {grad_mse}')

The MSE for a basic GradientBoosting Model is 2.4253464800253557


In [90]:
# SVR
svr = SVR()

svr.fit(X_train, y_train)

svr_pred = svr.predict(X_test)

svr_mse = mean_squared_error(y_test, svr_pred)

print(f'Basic SVR model MSE is {svr_mse}')

Basic SVR model MSE is 7.140934567092472


**Without any model tuning, random forest performs best, slightly above ada. Model tuning would be next to see which model and hyperparameter selection works best.**

In [91]:
best_model = rf
best_mse = rf_mse